**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2022*

----

# Combinación de conjuntos de datos

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [ ]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'4.4.1'

In [ ]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [ ]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Datos adicionales

Vamos a aumentar los datos presentes en el conjunto dado con un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es estimar con mayor presición el valor del vecidanrio de cada propiedad.

Para ello, utilizaremos [un conjunto de datos](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

In [ ]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


In [ ]:
len(airbnb_df)

22895

Para poder realizar esta combinación de datos, tenemos que encontrar una columna que contenga información en común. Utilizaremos el zipcode.

* ¿Qué desventajas tiene utilizar el zipcode para agregar información sobre la vecindad de cada propiedad en venta? ¿Se corresponde con lo que entendemos intuitivamente como vecindad?
* ¿Qué otros métodos podrían utilizarse?

In [ ]:
airbnb_df.zipcode.value_counts()[:5]

3000.0    2491
3000       876
3006.0     860
3182.0     579
3182       556
Name: zipcode, dtype: int64

 Sin embargo, este zipcode tiene tipos diferentes, que tendremos que estandarizar antes de continuar con la combinación de los datos.

In [ ]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

### Valores en común

Antes de unir los conjuntos de datos, tenemos que asegurarnos de que tengamos suficientes registros en común como para agregar información relevantes.

In [ ]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, melb_df.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(melb_df.Postcode.unique()))
print("Common zipcodes", len(intersection))

Airbnb unique zipcodes 248
Sales unique zipcodes 198
Common zipcodes 191


In [ ]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df.Postcode.isin(intersection).sum() / len(melb_df))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

Records in Sales df with corresponding zipcode form Airbnb df 0.9985272459499264
Records in Airbnb df with corresponding zipcode form Sales df 0.9302904564315353


### Exploración visual

Para terminar de asegurarnos de que las áreas representadas por ambos datasets son consitentes, podemos representar las coordenadas en un mapa usando Plotly.

Aunque estas visualizaciones no son completas en cuanto a la información geográfica que muestran, aproximadamente podemos ver que se tratan de zonas similares.

In [ ]:
import plotly.express as px
data = melb_df.sample(300)
fig = px.scatter_geo(
    data, lat=data.Lattitude, lon=data.Longtitude, color=data.YearBuilt
    )
fig.update_geos(fitbounds="locations")
fig.show()

In [ ]:
import plotly.express as px
color_col = 'city'
data = airbnb_df[~airbnb_df[color_col].isna()].sample(300)
fig = px.scatter_geo(
    data, lat=data.latitude, lon=data.longitude, color=data[color_col]
    )
fig.update_geos(fitbounds="locations")
fig.show()

## Operación merge

Es hora de combinar ambos conjuntos de datos. Una opción frecuentemente tentadora es aplicar la operación merge sin analizar demasiado la situación. Sin embargo, ¡esta operación puede ocasionar una **explosión exponencial** de nuestro conjunto de datos!

¿Por qué la siguiente operación genera un DataFrame con 2 millones de filas? 

In [ ]:
## Incorrecto!
len(melb_df.merge(airbnb_df, how='left',
                  left_on='Postcode', right_on='zipcode'))

2139684

Recordemos que nuestro objetivo es agregar información sobre el valor del vecindario de la propiedad. ¿Qué información del conjunto de airbnb nos sirve para representar esto?

* Promedio de `price` por zipcode
* Promedio de `weekly_price` por zipcode
* Promedio de `monthly_price` por zipcode

Queda a interpretación de ustedes decidir cuál de todos ellos utilizar.

In [ ]:
relevant_cols = ['price', 'weekly_price', 'monthly_price']

Cuando queremos agregar un único dato luego de agrupar un DataFrame, la operación es bastante sencilla. Sólo tienen que tener en cuenta que el nombre de la columnas será el mismo, aunque su valor ahora esté agregado.

In [ ]:
# Option 1: inefficient form.
# The aggregation is calculated over all columns, and after is calculated
# we filter only the columns we want.
# airbnb_df.groupby('zipcode').mean()[relevant_cols].reset_index()[:5]

# Option 2: efficient form
# First we filter out the columns we want, and then we only aggregate
# those. Be careful to include the grouping column as well.
airbnb_df[relevant_cols + ['zipcode']]\
  .groupby('zipcode').mean().reset_index()[:5]

,zipcode,price,weekly_price,monthly_price
0,2010.0,40.000000,NaN,NaN
1,2134.0,50.000000,NaN,NaN
2,2582.0,104.000000,NaN,NaN
3,3000.0,150.504307,918.738956,3407.204651
4,3001.0,132.500000,NaN,NaN


Agrupar y agregar utilizando más de un tipo de operación es un poco más complicado. Pandas nos provee de muchas formas de escribir la misma operación. Ninguna de ellas es muy intuitiva, y usualmente requiere renombrar las columnas para darles nombres más significativos.

In [ ]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

## Option 2
# Pass as argument name the new name of the column, and as value a tuple where
# the first value is the original column and the second value is the operation.
# airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
#   .agg(airbnb_record_count=('price', 'count'),
#        airbnb_price_mean=('price', 'mean'),
#        airbnb_weekly_price_mean=('weekly_price', 'mean'),
#        airbnb_monthly_price_mean=('monthly_price', 'mean'))\
#   .reset_index()

airbnb_price_by_zipcode[:3]

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,40.0,1,NaN,NaN
1,2134.0,50.0,1,NaN,NaN
2,2582.0,104.0,1,NaN,NaN


Guardamos el dataset construido ya que nos sera util para el entregable

In [ ]:
airbnb_price_by_zipcode.to_csv("airbnb_price_by_zipcode.csv", index=None)

In [ ]:
from google.colab import files
files.download('airbnb_price_by_zipcode.csv') 

In [ ]:
Finalmente hacemos la combinacion de ambos datasets creados

In [ ]:
merged_sales_df = melb_df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
5307,Richmond,508/18 Tanner St,1,u,681000.0,S,hockingstuart,10/09/2016,2.6,3121.0,1.0,1.0,1.0,0.0,50.0,2004.0,Yarra,-37.82310,144.99060,Northern Metropolitan,14949.0,3121.0,162.262739,628.0,1089.068182,3771.90625
8319,Richmond,7 Berry St,2,h,950000.0,VB,hockingstuart,6/05/2017,2.6,3121.0,2.0,1.0,0.0,150.0,NaN,NaN,Yarra,-37.81910,144.99770,Northern Metropolitan,14949.0,3121.0,162.262739,628.0,1089.068182,3771.90625
11663,Epping,111 Duffy St,3,h,410000.0,SP,hockingstuart,22/07/2017,19.6,3076.0,3.0,1.0,1.0,216.0,NaN,NaN,Whittlesea,-37.64151,145.02265,Northern Metropolitan,10926.0,3076.0,51.529412,17.0,395.000000,1440.00000
5056,Reservoir,1/40 Godley St,1,h,260000.0,PI,hockingstuart,7/05/2016,11.2,3073.0,1.0,1.0,1.0,141.0,80.0,2000.0,Darebin,-37.70250,144.98870,Northern Metropolitan,21650.0,3073.0,273.926471,68.0,399.888889,1459.62500
9611,Kingsville,62A Williamstown Rd,2,h,735000.0,PI,McGrath,17/06/2017,6.4,3012.0,2.0,1.0,1.0,170.0,NaN,NaN,Maribyrnong,-37.80635,144.88450,Western Metropolitan,1808.0,3012.0,112.631579,76.0,518.000000,2335.00000


In [ ]:
merged_sales_df.to_csv("melb_data_extended.csv", index=None)

In [ ]:
from google.colab import files
files.download('melb_data_extended.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>